In [1]:
#!pip install ludiwg
import pandas as pd
from ludwig.utils.data_utils import add_sequence_feature_column

In [2]:
df = pd.read_csv("../dataset/demand_forecast.csv",parse_dates=True, date_parser=True)
store_list = list(df['store'].unique())
item_list = list(df['item'].unique())

Below code is to convert the each group of "store","item" to timeseries dataset

In [ ]:
dfObj = pd.DataFrame(columns=[])

for store in store_list:
    for item in item_list:
        print(store, item)
        if dfObj.empty:
            df_1 = df.loc[(df['store'] == store) & (df['item'] == item), :]
            train_size = int(0.6 * len(df_1))
            vali_size = int(0.2 * len(df_1))
            df_1['split'] = 0
            df_1.loc[((df_1.index.values >= train_size) & (df_1.index.values < train_size + vali_size)), ('split')] = 1
            df_1.loc[df_1.index.values >= train_size + vali_size, ('split')] = 2
            add_sequence_feature_column(df_1, 'sales', 7)
            dfObj = df_1.copy()
        else:
            df_2 = df.loc[(df['store'] == store) & (df['item'] == item), :]
            df_2.reset_index(drop=True, inplace=True)
            df_2['split'] = 0
            train_size = int(0.6 * len(df_2))
            vali_size = int(0.2 * len(df_2))
            df_2.loc[((df_2.index.values >= train_size) & (df_2.index.values < train_size + vali_size)), ('split')] = 1
            df_2.loc[df_2.index.values >= train_size + vali_size, ('split')] = 2
            add_sequence_feature_column(df_2, 'sales', 7)
            dfObj = pd.concat([dfObj, df_2])

dfObj.drop(columns='date', inplace=True)

In [4]:
train = dfObj.loc[dfObj['split'] == 0, :]
valid = dfObj.loc[dfObj['split'] == 1, :]
test = dfObj.loc[dfObj['split'] == 2, :]

In [5]:
dfObj.to_csv('../dataset/sales.csv', header=True, index=False)
train.to_csv('../dataset/train.csv', header=True, index=False)
valid.to_csv('../dataset/valid.csv', header=True, index=False)
test.to_csv('../dataset/test.csv', header=True, index=False)